In [1]:
import numpy as np
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout,Flatten
from sklearn.model_selection import train_test_split
from nltk.corpus import twitter_samples
import nltk
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimoa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\kimoa\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
tweets = positive_tweets + negative_tweets
labels = np.append(np.ones(len(positive_tweets)), np.zeros(len(negative_tweets)))
stopwords_english = stopwords.words('english')

In [3]:
def clean_tweets(tweet):
    # Remove URLs and mentions
    tweet = re.sub(r"http\S+|www\S+|https\S+|\@\w+", "", tweet)
    # Remove non-alphanumeric characters and convert to lowercase
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet).lower()
    # Tokenize the tweet into words
    words = tweet.split()
    # Remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    # Convert all words to string
    cleaned_tweet = ' '.join(str(word) for word in words)
    return cleaned_tweet


In [4]:
cleaned_tweets = []
for tweet in tweets:
    cleaned_tweets.append(clean_tweets(tweet))

# Convert the list of words to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweets)
sequences = tokenizer.texts_to_sequences(cleaned_tweets)

# Pad the sequences to a fixed length
max_len = 30
padded_sequences = np.zeros((len(sequences), max_len), dtype=int)
for i, sequence in enumerate(sequences):
    j = max_len - len(sequence)
    padded_sequence_str = ' '.join(map(str, sequence[:max_len-j]))
    padded_sequences[i, j:] = np.array(list(map(int, padded_sequence_str.split())))

In [5]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 32
dropout_rate = 0.5

In [6]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
batch_size = 64
epochs = 10

In [30]:
# Reshape the ground truth labels
y_train = np.reshape(y_train, (y_train.shape[0], 1))

# Train the model
# history = model.fit(padded_sequences, y_train, batch_size=64, epochs=10)

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
125/125 [==============================] - 2s 13ms/step - loss: 0.6822 - accuracy: 0.6022 - val_loss: 0.6578 - val_accuracy: 0.7180
Epoch 2/10
125/125 [==============================] - 1s 9ms/step - loss: 0.5742 - accuracy: 0.8145 - val_loss: 0.5425 - val_accuracy: 0.7430
Epoch 3/10
125/125 [==============================] - 1s 10ms/step - loss: 0.4095 - accuracy: 0.8756 - val_loss: 0.4883 - val_accuracy: 0.7605
Epoch 4/10
125/125 [==============================] - 1s 10ms/step - loss: 0.2966 - accuracy: 0.9124 - val_loss: 0.4818 - val_accuracy: 0.7565
Epoch 5/10
125/125 [==============================] - 1s 10ms/step - loss: 0.2220 - accuracy: 0.9362 - val_loss: 0.4892 - val_accuracy: 0.7545
Epoch 6/10
125/125 [==============================] - 1s 10ms/step - loss: 0.1717 - accuracy: 0.9521 - val_loss: 0.5054 - val_accuracy: 0.7485
Epoch 7/10
125/125 [==============================] - 1s 10ms/step - loss: 0.1371 - accuracy: 0.9621 - val_loss: 0.5243 - val_accuracy: 0.7450


In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert the list of words to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweets)
sequences = tokenizer.texts_to_sequences(cleaned_tweets)

# Pad the sequences to a fixed length
max_len = 30
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

# Define the Keras model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, labels, batch_size=64, epochs=epochs)


Epoch 1/10
157/157 [==============================] - 2s 8ms/step - loss: 0.6780 - accuracy: 0.6060
Epoch 2/10
157/157 [==============================] - 1s 9ms/step - loss: 0.5373 - accuracy: 0.8152
Epoch 3/10
157/157 [==============================] - 1s 9ms/step - loss: 0.3725 - accuracy: 0.8792
Epoch 4/10
157/157 [==============================] - 1s 8ms/step - loss: 0.2699 - accuracy: 0.9161
Epoch 5/10
157/157 [==============================] - 1s 9ms/step - loss: 0.2024 - accuracy: 0.9363
Epoch 6/10
157/157 [==============================] - 1s 8ms/step - loss: 0.1571 - accuracy: 0.9527
Epoch 7/10
157/157 [==============================] - 1s 9ms/step - loss: 0.1267 - accuracy: 0.9628
Epoch 8/10
157/157 [==============================] - 1s 9ms/step - loss: 0.1052 - accuracy: 0.9693
Epoch 9/10
157/157 [==============================] - 1s 8ms/step - loss: 0.0896 - accuracy: 0.9738
Epoch 10/10
157/157 [==============================] - 1s 8ms/step - loss: 0.0779 - accuracy: 0.9767

In [13]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07036126405000687
Test accuracy: 0.9779999852180481


In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert the list of words to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(cleaned_tweets)
sequences = tokenizer.texts_to_sequences(cleaned_tweets)

# Pad the sequences to a fixed length
max_len = 30
padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

# Define the Keras model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, labels, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))


Epoch 1/10
157/157 [==============================] - 3s 11ms/step - loss: 0.6767 - accuracy: 0.5978 - val_loss: 0.6118 - val_accuracy: 0.8200
Epoch 2/10
157/157 [==============================] - 1s 9ms/step - loss: 0.5330 - accuracy: 0.8173 - val_loss: 0.3997 - val_accuracy: 0.8785
Epoch 3/10
157/157 [==============================] - 1s 9ms/step - loss: 0.3714 - accuracy: 0.8745 - val_loss: 0.2824 - val_accuracy: 0.9155
Epoch 4/10
157/157 [==============================] - 1s 9ms/step - loss: 0.2712 - accuracy: 0.9140 - val_loss: 0.2130 - val_accuracy: 0.9365
Epoch 5/10
157/157 [==============================] - 1s 9ms/step - loss: 0.2048 - accuracy: 0.9376 - val_loss: 0.1648 - val_accuracy: 0.9520
Epoch 6/10
157/157 [==============================] - 1s 9ms/step - loss: 0.1601 - accuracy: 0.9505 - val_loss: 0.1326 - val_accuracy: 0.9615
Epoch 7/10
157/157 [==============================] - 1s 9ms/step - loss: 0.1290 - accuracy: 0.9615 - val_loss: 0.1105 - val_accuracy: 0.9685
Epoch

In [15]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07320137321949005
Test accuracy: 0.9769999980926514
